In [7]:
import pygame
import numpy as np
import math
from Index import Graphics, Vehicle, Ultrasonic

MapDimensions = (720, 720)

In [8]:
def distance(point1, point2):
    point1 = np.array(point1)
    point2 = np.array(point2)
    return np.linalg.norm(point1-point2)

In [9]:
class Vehicle:
    def __init__(self, startpos, width):
        self.m2p = 3779.52
        self.width = width

        self.x = startpos[0]
        self.y = startpos[1]
        self.heading = 0

        self.vl = 0.01*self.m2p
        self.vr = 0.01*self.m2p
        
        self.maxspeed = 0.02*self.m2p
        self.minspeed = 0.01*self.m2p

        self.minObsDistance = 100
        self.countDown = 5

    def avoidObsticles(self, pointCloud, dt):
        closestObj = None
        dist = np.inf

        if(len(pointCloud) > 1):
            for point in pointCloud:
                if dist > distance([self.x, self.y], point):
                    dist = distance([self.x, self.y], point)
                    closestObj = (point, dist)

            if(closestObj[1] < self.minObsDistance and self.countDown > 0):
                self.countDown -= dt
                self.moveBackward()
            else:
                self.countDown = 5
                self.moveForward()

    def moveForward(self):
        self.vr = self.minspeed
        self.vl = self.minspeed

    def moveBackward(self):
        self.vr = -self.minspeed
        self.vl = -self.minspeed/2

    def kinematics(self, dt):
        self.x += ((self.vr + self.vl)/2) * math.cos(self.heading)*dt
        self.y -= ((self.vr + self.vl)/2) * math.sin(self.heading)*dt
        self.heading += ((self.vr - self.vl)/self.width)*dt

        if(self.heading>2*math.pi or self.heading<-2*math.pi):
            self.heading = 0

        self.vr = max(min(self.maxspeed, self.vr), self.minspeed)
        self.vl = max(min(self.maxspeed, self.vl), self.minspeed)

In [10]:
class Graphics:
    def __init__(self, dimentions, VehicleImg, MapImg):
        pygame.init()
        self.black = (0, 0, 0)
        self.white = (255, 255, 255)
        self.green = (0, 255, 0)
        self.blue = (0, 0, 255)
        self.red = (255, 0, 0)
        self.yellow = (255, 255, 0)
        
        self.vehicle = pygame.image.load(VehicleImg)
        self.map = pygame.image.load(MapImg)

        self.width, self.height = dimentions

        pygame.display.set_caption("Obsticle Avoidance")
        self.map = pygame.display.set_mode((self.width, self.height))
        self.map.blit(pygame.transform.scale(MapImg, ((self.width/1.3 - self.width/5), self.height)), (self.width/5, 0))

    def drawVehicle(self, x, y, heading):
        rotated = pygame.transform.rotozoom(self.vehicle, math.degrees(heading), 1)
        rect = rotated.get_rect(center=(x, y))
        self.map.blit(rotated, rect)
    
    def drawSensorData(self, pointCloud):
        for point in pointCloud:
            pygame.draw.circle(self.map, self.red, point, 3, 0)

class Ultrasonic:
    def __init__(self, sensorRange, map):
        self.sensorRange = sensorRange
        self.mapWidth, self.mapHeight = map.get_size()
        self.map = map

    def senseObstacles(self, x, y, heading):
        obstacles = []
        x1, y1 = x, y
        startAngle = heading - self.sensorRange[1]
        finishAngle = heading - self.sensorRange[1]
        for angle in np.linspace(startAngle, finishAngle, 10, False):
            x2 = x + self.sensorRange * math.cos(angle)
            y2 = y + self.sensorRange * math.sin(angle)

            for i in range(0, 100):
                u = i/100
                x = int(x2 *u + x1 * (1-u))
                y = int(y2 *u + y1 * (1-u))

                if(0 < x < self.mapWidth and 0 < y < self.mapHeight):
                    color = self.map.get_at((x, y))
                    self.map.set_at((x, y), (0, 208, 255))
                    if(color[0], color[1], color[2]) == (0, 0, 0):
                        obstacles.append([x, y])
                        break
        return obstacles
    

In [11]:
# Initialize pygame
pygame.init()

# Set up the game window
window_width = 1300
window_height = 650
gfx = Graphics(MapDimensions, "road_3.png")
start_positions = [(100, 300), (200, 500), (300, 400), (400, 200), (500, 300), (600, 500),(300, 300), (700, 500), (700, 100), (400, 400), (400, 500), (500, 600)]
vehicle_images = ["Vehicle.png", "Vehicle.png", "Vehicle.png", "Vehicle.png", "Vehicle.png", "Vehicle.png","Vehicle.png", "Vehicle.png", "Vehicle.png", "Vehicle.png", "Vehicle.png", "Vehicle.png"]

vehicles = []
for pos, image in zip(start_positions, vehicle_images):
    vehicle = Vehicle(pos, 0.01 * 3779.52, image)
    vehicles.append(vehicle)

# Create the ultrasonic sensor object
sensorRange = 200, math.radians(30)
ultraSonic = Ultrasonic(sensorRange, gfx.map)

# Initialize time variables for simulation loop

# bg_image = pygame.image.load('road_1.png')
# window = pygame.display.set_mode((window_width, window_height))
# pygame.display.set_caption("My Game")

# Game loop
running = True
while running:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # Get the mouse position and print it
            mouse_x, mouse_y = pygame.mouse.get_pos()
            print(f"Mouse clicked at ({mouse_x}, {mouse_y})")

    dt = 0
    lastTime = pygame.time.get_ticks()

    # Update game state
    gfx.map.blit(gfx.MapImg, (0, 0))

    for vehicle in vehicles:
        if(vehicle.x > window_width or vehicle.x < 0 or vehicle.y > window_height or vehicle.y < 0):
            vehicles.remove(vehicle)
            break
        vehicle.kinematics(dt)
        for other_vehicle in vehicles:
            if vehicle != other_vehicle:  # Make sure not to check a vehicle with itself
                if vehicle.check_collision(other_vehicle):
                    vehicles.remove(vehicle)
                    vehicles.remove(other_vehicle)
                    print("collision")
                    # You might want to add more sophisticated collision response here
        gfx.drawVehicle(vehicle.x, vehicle.y, vehicle.heading, vehicle.image)  # Pass image to drawVehicle
        pointCloud = ultraSonic.senseObstacles(vehicle.x, vehicle.y, vehicle.heading)
        vehicle.avoidObstacles(pointCloud, dt, vehicles)
        gfx.drawSensorData(pointCloud)

    pygame.display.update()
    # Render the game
    window.fill((0, 0, 0))  # Fill the window with black
    # Draw everything here
    window.blit(pygame.transform.scale(bg_image, ((window_width/1.3 - window_width/5), window_height)), (window_width/5, 0))
    
    pygame.draw.line(window, (255, 255, 255), (window_width/5, 0), (window_width/5, window_height), 5)
    pygame.draw.line(window, (255, 255, 255), (window_width/1.3, 0), (window_width/1.3, window_height), 5)
    pygame.draw.line(window, (255, 255, 255), (window_width/1.3, window_height/2), (window_width, window_height/2), 5)

    pygame.draw.rect(window, (255, 255, 255), (10, 15, window_width/5.5, window_height/3.5), 5)
    pygame.draw.rect(window, (255, 255, 255), (10, (15+(window_height/3)), window_width/5.5, window_height/3.5), 5)
    pygame.draw.rect(window, (255, 255, 255), (10, (15+((window_height/3)+(window_height/3))), window_width/5.5, window_height/3.5), 5)

    pygame.draw.circle(window, (0,0,0), (468, 636), 15)

    pygame.display.flip()  # Update the display

# Quit the game
pygame.quit()


TypeError: __init__() missing 1 required positional argument: 'MapImg'